# How to get the Grassroots data directly to your machine and play with it.


###  Studies with plot data:
#### ["https://grassroots.tools/fieldtrial/plots/603e3e9502700f7faf25dfb4")](https://grassroots.tools/fieldtrial/plots/603e3e9502700f7faf25dfb4).
#### ["https://grassroots.tools/fieldtrial/plots/60955bb402700f084b013da4")](https://grassroots.tools/fieldtrial/plots/60955bb402700f084b013da4).

In [ ]:
import json
import requests    #HTTP library

server_url = "http://localhost:2000/grassroots/public_backend"
#server_url = "http://10.0.152.67/grassroots/public_backend"
#server_url = "https://grassroots.tools/public_backend"

#----------------------------------------------------------------------------------
#Request to get study 
def get_plot(id):
    plot_request = {
            "services": [{
                "so:name": "Search Field Trials",
                "start_service": True,
                "parameter_set": {
                    "level": "advanced",
                    "parameters": [{
                        "param": "ST Id",
                        "current_value": id
                    }, {
                        "param": "Get all Plots for Study",
                        "current_value": True
                    }, {
                        "param": "ST Search Studies",
                        "current_value": True
                    }]
                }
            }]
        }
    res = requests.post(server_url, data=json.dumps(plot_request))
    return json.dumps(res.json())
#----------------------------------------------------------------------------------

json_study   = get_plot("603e3e9502700f7faf25dfb4")   #  Rothamsted Harvest 2019
json_study   = get_plot("60955bb402700f084b013da4") 

single_study = json.loads(json_study)   #Deserialising data

print(single_study['results'][0]['results'][0]['data']['so:name'])          # 1)Basic description of study
print(single_study['results'][0]['results'][0]['data']['so:description'])    
print("\n")
#print(single_study['results'][0]['results'][0]['data']['phenotypes'])   # 2)Description of each phenotype 
#print(single_study['results'][0]['results'][0]['data']['phenotypes']['HI_CalcGbSamp_%']['definition']['trait'])
#print("\n")
#print(single_study['results'][0]['results'][0]['data']['phenotypes']['GrnYld85_Fh_tha']['definition']['trait'])

#print(single_study['results'][0]['results'][0]['data']['plots'])       #  3)Observation measured in each plot


#print(single_study['results'][0]['results'][0]['data']['phenotypes']['GMoi_E_pct']['definition']['trait'])
#print(single_study['results'][0]['results'][0]['data']['phenotypes']['PH_M_cm']['definition']['trait'])

## Tidying up

### Extract and organise some basic information about the phenotypes 

In [ ]:
import nbimporter
from src.grassroots_requests import get_plot   #Put request into a function

import json

#----------------------------------------------------------------------------------
def dict_phenotypes(pheno, plots):

    names  = []
    traits = []
    
    for key in pheno:
        # From the phenotypes data, just take the names and its trait description.
        names.append(key)
        traits.append(pheno[key]['definition']['trait']['so:name'])

    phenoDict = dict(zip(names, traits))
    
    # General loop to check for non-numeric values.
    for j in range(len(plots)):
        if ( 'discard' in plots[j]['rows'][0] ):
            pass
        if ('observations' in plots[j]['rows'][0]):            
            for k in range(len(plots[j]['rows'][0]['observations'])):
                if ('raw_value' in plots[j]['rows'][0]['observations'][k]):
                    rawValue = plots[j]['rows'][0]['observations'][k]['raw_value']
                if ('corrected_value' in plots[j]['rows'][0]['observations'][k]):
                    rawValue = plots[j]['rows'][0]['observations'][k]['corrected_value']
                if ( type(rawValue) == str):
                    #print(rawValue)
                    name = plots[j]['rows'][0]['observations'][k]['phenotype']['variable']
                    if( name in phenoDict.keys() ):
                        print("Remove pheonotypes with non-numeric value: ", phenoDict[name])
                        del phenoDict[name]

    return phenoDict
#----------------------------------------------------------------------------------


json_study   = get_plot("603e3e9502700f7faf25dfb4") 
#json_study   = get_plot("60955bb402700f084b013da4") 

single_study = json.loads(json_study)   #Deserialising data

plots      = single_study['results'][0]['results'][0]['data']['plots']
phenotypes = single_study['results'][0]['results'][0]['data']['phenotypes']

traits = dict_phenotypes(phenotypes, plots)

print(single_study['results'][0]['results'][0]['data']['so:name'])    
print("Number of phenotypes in current study: ", len(traits))
print("\n")
#for item in traits:
#    print(f'{item}: \t {traits[item]}')
    
print("Total number of plots: ", len(single_study['results'][0]['results'][0]['data']['plots']))  
print("\n")


print(single_study['results'][0]['results'][0]['data']['plots'][1]['rows'][0] )
print(single_study['results'][0]['results'][0]['data']['plots'][2]['rows'][0]['observations'][0]['phenotype'])
print(single_study['results'][0]['results'][0]['data']['plots'][2]['rows'][0]['observations'][0]['raw_value'])    
print(single_study['results'][0]['results'][0]['data']['plots'][3]['rows'][0]['observations'][0]['phenotype'])
print(single_study['results'][0]['results'][0]['data']['plots'][3]['rows'][0]['observations'][0]['raw_value'])    



## Loop through plot array to create matrix

### use Seaborn package to create heatmap

In [ ]:
import json
import nbimporter
from src.grassroots_requests import get_plot
from src.grassroots_plots    import dict_phenotypes

from src.grassroots_plots    import create_matrices
from src.grassroots_plots    import seaborn_plot

json_study   = get_plot("603e3e9502700f7faf25dfb4") 
#json_study   = get_plot("60955bb402700f084b013da4") 
single_study = json.loads(json_study)   #Deserialising data


plots      = single_study['results'][0]['results'][0]['data']['plots']
phenotypes = single_study['results'][0]['results'][0]['data']['phenotypes']
total_rows = single_study['results'][0]['results'][0]['data']['num_rows']
total_cols = single_study['results'][0]['results'][0]['data']['num_columns']

traits     = dict_phenotypes(phenotypes, plots)

phenotype_selected = "HI_CalcGbSamp_%" 
#phenotype_selected = "GrnYld85_Fh_tha" 
phenotype_selected = list(phenotypes.keys())[0]
title = traits[phenotype_selected]
print(phenotype_selected, title)

matrices  = create_matrices(plots, phenotypes, phenotype_selected, total_rows, total_cols)

rows       = matrices[0] 
columns    = matrices[1]
raw_values = matrices[2]
title = matrices[3]
units = matrices[4]

matrix   = raw_values.reshape(rows,columns)
#print(matrix)

colormap ="seagreen"
#colormap ="xkcd:copper"
#colormap="#a275ac"
seaborn_plot(matrix, title, units, phenotype_selected, colormap)

## Inteactive heatmap using Plotly package

In [5]:
import json
import nbimporter
from src.grassroots_requests import get_plot
from src.grassroots_plots    import dict_phenotypes

from src.grassroots_plots    import create_matrices
from src.grassroots_plots    import plotly_plot

json_study   = get_plot("603e3e9502700f7faf25dfb4") 
#json_study   = get_plot("60955bb402700f084b013da4") 
single_study = json.loads(json_study)   #Deserialising data


plots      = single_study['results'][0]['results'][0]['data']['plots']
phenotypes = single_study['results'][0]['results'][0]['data']['phenotypes']
total_rows = single_study['results'][0]['results'][0]['data']['num_rows']
total_cols = single_study['results'][0]['results'][0]['data']['num_columns']

traits     = dict_phenotypes(phenotypes, plots)

phenotype_selected = "HI_CalcGbSamp_%" 
#phenotype_selected = "GrnYld85_Fh_tha" 
phenotype_selected = list(phenotypes.keys())[0]
title = traits[phenotype_selected]
print(phenotype_selected, title)

matrices  = create_matrices(plots, phenotypes, phenotype_selected, total_rows, total_cols)

rows       = matrices[0] 
columns    = matrices[1]
raw_values = matrices[2]
title = matrices[3]
units = matrices[4]
accession  = matrices[5]

accession   = accession.reshape(rows,columns)
#print(matrix)
#print(accession)

#plotly_plot(raw_values, accession, title, units)


HI_CalcGbSamp_% Harvest index


In [1]:
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

import dash_bootstrap_components as dbc          
from dash.exceptions import PreventUpdate        

#################################################
import json
import nbimporter
from src.grass_plots import get_all_fieldtrials     
from src.grass_plots import get_plot
from src.grass_plots import dict_phenotypes
from src.grass_plots import numpy_data
from src.grass_plots import treatments
from src.grass_plots import plotly_plot
import operator                                   
import numpy as np                               
import requests 
#################################################


all_studies  = get_all_fieldtrials()
all_studies = json.loads(all_studies)

studiesIDs = []
names      = []
for i in range(len(all_studies['results'][0]['results'])):
        uuid  = all_studies['results'][0]['results'][i]['data']['_id']['$oid']
        name = all_studies['results'][0]['results'][i]['data']['so:name']

        if 'phenotypes' in all_studies['results'][0]['results'][i]['data']:
            studiesIDs.append(uuid)
            names.append(name)

#studiesIDs.remove('619e159b87a279348474145b')           # DFW Academic Toolkit RRes, Harvest 2021   
#names.remove('DFW Academic Toolkit RRes, Harvest 2021') 
#studiesIDs.remove('6225dfde93b7641e4b5acb85')           #  NIAB CSSL AB Glasshouse exp 
#names.remove('NIAB CSSL AB Glasshouse exp ')    
studiesIDs.remove('5dd8009ade68e75a927a8274')  #  
names.remove('1st vs 3rd wheat take-all resistance trial')   #


optionsNames = [{'label': names[i], 'value':studiesIDs[i]} for i in range(len(names))]

optionsNames.sort(key=operator.itemgetter('label'))  # Sort list of dictionaries by key 'label'

           
#app = Dash(__name__)
app = JupyterDash(__name__)   # , external_stylesheets=[dbc.themes.BOOTSTRAP]


app.layout = html.Div([

      html.Div(children=[
      html.Label(['List of studies:'],style={'font-weight': 'bold', "text-align": "left"}),

      dcc.Dropdown(id='DROPDOWN1',
          options = optionsNames,
          value   = optionsNames[0]['value'],
          searchable = True,
          style={'width':"100%"},
          #search_value='',
      ),
      html.Div(id='STUDY'),
      html.Br(),

    ]),
    
    html.Div(children=[
        html.Label(['Select phenotype:'],style={'font-weight': 'bold', "text-align": "left"}),
        
        dcc.Dropdown( id='DROPDOWN2',
              options = [],
              #value   = 'SpkPop_CalcGbSamp_m2', 
              searchable = True,
              style={'width':"100%"},
          )
    
    ]),

])
#-------------------------------------------------
@app.callback(
    Output('STUDY', 'children'),
    Input('DROPDOWN1', 'value')
)

def print_uuid(uuid):

    if uuid is None:
        raise PreventUpdate

    return ('Study uuid: {} '.format(uuid))

########################### **update dropdown 2 (List of phenotypes)** ###########################
@app.callback(
    [Output('DROPDOWN2', 'options'),
     Output('DROPDOWN2', 'value') ],
     Input( 'DROPDOWN1', 'value') )

def update_dropdown_menu(uuid):

    if uuid is None:
        raise PreventUpdate

    single_study = get_plot(uuid)
    study_json   = json.loads(single_study)

    studies_ids =[]

    if 'phenotypes' in study_json['results'][0]['results'][0]['data']:
        studies_ids.append(uuid)
        #print("Study has phenotypes", studies_ids)


    plot_data         = study_json['results'][0]['results'][0]['data']['plots']
    phenotypes        = study_json['results'][0]['results'][0]['data']['phenotypes']

    dictTraits = dict_phenotypes(phenotypes, plot_data)

    phenoKeys   = list(dictTraits.keys())
    phenoValues = list(dictTraits.values())

    options = [{'label': phenoValues[i], 'value':phenoKeys[i]} for i in range(len(phenoKeys))]

    value   = list(dictTraits.keys())[0]
    return options, value



if __name__ == "__main__":
    app.run_server(mode='inline', port=8052)
    

Remove: Anthesis time
Remove: Maturity time
